#Libraries

In [0]:
%pip install xgboost
%pip install skforecast

In [0]:
%restart_python

In [0]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

import xgboost as xgb
from skforecast.recursive._forecaster_recursive import ForecasterRecursive
from skforecast.preprocessing import RollingFeatures
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import TimeSeriesFold
from skforecast.model_selection._search import random_search_forecaster 

from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from scipy.stats import randint, uniform

#Customized Functions

In [0]:
def model_evaluate(time_series, time_series_pred, split_date = '2014-01-01'):

    y_train = time_series.loc[time_series.index < split_date]
    y_train = y_train[time_series_pred.index.min():]
    y_train_pred = time_series_pred.loc[time_series_pred.index < split_date]
    y_test = time_series.loc[time_series.index >= split_date]
    y_test_pred = time_series_pred.loc[time_series_pred.index >= split_date]
    
    print("", end="\t")
    print("train", end="\t\t\t")
    print("test", end="\n")

    print("r2", end="\t")
    print(r2_score(y_train, y_train_pred), end="\t")
    print(r2_score(y_test, y_test_pred))

    print("rmse", end="\t")
    print(np.sqrt(mean_squared_error(y_train, y_train_pred)), end="\t")
    print(np.sqrt(mean_squared_error(y_test, y_test_pred)))

    print("mae", end="\t")
    print(mean_absolute_error(y_train, y_train_pred), end="\t")
    print(mean_absolute_error(y_test, y_test_pred))

In [0]:
def forecast_evaluate(y_test, y_test_pred):
    
    print("r2: ", r2_score(y_test, y_test_pred))
    print("rmse: ", np.sqrt(mean_squared_error(y_test, y_test_pred)))
    print("mae: ", mean_absolute_error(y_test, y_test_pred))

In [0]:
# Function to plot original data and predictions
def plot_predictions(df, preds, test_size = 90, target = "unit_sales"):
    """
    Parameters:
    df (pd.DataFrame): The original time series data grouped by date.
    preds (np.Array): The predicted values for the test period.
    test_size (int): Number of test observations used for evaluation. Default is 12.
    """
    # Create the plot
    plt.figure(figsize=(20, 5))
    sns.lineplot(x=df.date, y=df[target], label='Original Data')
    sns.lineplot(x=df.date[-test_size:], y=preds, label='Predictions')

    # Add labels and title
    plt.xticks(rotation = 45)
    plt.xlabel('Date')
    plt.ylabel('Values')
    plt.title('Forecast vs Actual')
    plt.legend()
    plt.show()

#Import Dataset

In [0]:
df = spark.table("workspace.timeseries.train2").toPandas()
df_holidays = spark.table("workspace.timeseries.holidays").toPandas()
#df_items = spark.table("workspace.timeseries.items").toPandas()
#df_oil = spark.table("workspace.timeseries.oil").toPandas()
#df_stores = spark.table("workspace.timeseries.stores").toPandas()
df_transactions = spark.table("workspace.timeseries.transactions").toPandas()

In [0]:
#convert date to datetime
df.date = pd.to_datetime(df.date)
df_holidays.date = pd.to_datetime(df_holidays.date)
#df_oil.date = pd.to_datetime(df_oil.date)
df_transactions.date = pd.to_datetime(df_transactions.date)

In [0]:
df.head()
#it contains data from Guayas region only
#rows with unit_sales == 0 have been removed: if a product was not sold a specific day, this product has not a row for that day
#the data are between 2013-01-02 and 2014-03-31

#Dataset Construction

In [0]:
df.head()

##Time Series

In [0]:
#create the pandas series containing unit sales by date
time_series = df.groupby('date')['unit_sales'].sum()

#assign the daily frequence to sales_by_date
time_series = time_series.asfreq('D')

time_series.head(3)

In [0]:
#checking if there are any days without purchases
print(time_series[time_series.isnull()])
#yes, on christmas! 
#replacing it with 0
time_series = time_series.fillna(0)

In [0]:
time_series.head(3)

##Exogenous Features

Features extracted from the date

In [0]:
# Create dataframe with exogenous features from the time series
exog_feat_df = time_series.reset_index()

# Extract day, month, weekday, day_of_year from the date column
exog_feat_df['day'] = exog_feat_df['date'].dt.day
exog_feat_df['month'] = exog_feat_df['date'].dt.month
exog_feat_df['weekday'] = exog_feat_df['date'].dt.weekday
exog_feat_df["day_of_year"] = exog_feat_df["date"].dt.dayofyear

# Encode month, weekday and doy as cyclical features
exog_feat_df["month_sin"] = np.sin(2 * np.pi * exog_feat_df["month"] / 12)
exog_feat_df["month_cos"] = np.cos(2 * np.pi * exog_feat_df["month"] / 12)
exog_feat_df["weekday_sin"] = np.sin(2 * np.pi * exog_feat_df["weekday"] / 7)
exog_feat_df["weekday_cos"] = np.cos(2 * np.pi * exog_feat_df["weekday"] / 7)
exog_feat_df["doy_sin"] = np.sin(2 * np.pi * exog_feat_df["day_of_year"] / 365)
exog_feat_df["doy_cos"] = np.cos(2 * np.pi * exog_feat_df["day_of_year"] / 365)

# Create a temporal index for each row
exog_feat_df['temporal_idx'] = np.arange(len(exog_feat_df))

# Create a binary column indicating if the day is a weekend (Saturday or Sunday)
exog_feat_df['is_weekend'] = exog_feat_df['weekday'].isin([5, 6]).astype(int)

# Drop columns that are not needed for modeling
exog_feat_df = exog_feat_df.drop(columns=['unit_sales', 'month', 'weekday', 'day_of_year'])

# Set date as the index of the dataframe
exog_feat_df = exog_feat_df.set_index('date')
exog_feat_df.head(3)

Transactions

In [0]:
# Aggregate daily transactions and add as a feature to exog_feat_df
trans_by_date = df_transactions.groupby('date')['transactions'].sum()
trans_by_date = trans_by_date.asfreq('D')
trans_by_date = trans_by_date['2013-01-02':'2014-03-31']
trans_by_date = trans_by_date.fillna(0)

exog_feat_df['transactions'] = trans_by_date
exog_feat_df.head(3)

Holidays

In [0]:
# Select holidays dates for the Guayas region within the specified date range
guayas_holidays = df_holidays.loc[
    ((df_holidays.locale_name == "Guayaquil") | (df_holidays.locale_name == "Ecuador")) 
    & ((df_holidays.date > '2013-01-01') & (df_holidays.date <= '2014-03-31')), ['date']
]

# Add a column indicating that these dates are holidays
guayas_holidays['is_holiday'] = True

# Set the date as the index for the dataframe
guayas_holidays = guayas_holidays.set_index('date')

guayas_holidays.head(3)

In [0]:
# Merge exogenous features dataframe with holidays dataframe, aligning on date index
exog_feat_df = pd.merge(exog_feat_df, guayas_holidays, how='left', left_index=True, right_index=True)

# Fill missing values in is_holiday column with False
exog_feat_df.is_holiday = exog_feat_df.is_holiday.fillna(False)

# Convert boolean is_holiday column to binary (1 for holiday, 0 for non-holiday)
exog_feat_df.is_holiday = exog_feat_df.is_holiday.replace({True: 1, False: 0})

exog_feat_df.head()

Train and Test Split

In [0]:
split_date = '2014-01-01'

#time series train/test splitting
time_series_train = time_series.loc[time_series.index < split_date]
time_series_test = time_series.loc[time_series.index >= split_date]

#exogenous feature dataframe train/test splitting
exog_feat_df_train = exog_feat_df.loc[exog_feat_df.index < split_date]
exog_feat_df_test = exog_feat_df.loc[exog_feat_df.index >= split_date]

#ML

##Hyperparameter tuning

In [0]:
#Here the hyperparameter tuning is performed
#Since it took ~12 minutes to run, I ran it once and took note of the best parameter (they are listed 2 cells below)
#Just to make the whole notebook run fast, I reduced the number of iterations from 1000 to 10 and the lags grid to only 1 set of values

# Forecaster initialization
xgboost_model = xgb.XGBRegressor(random_state=123)

# Set up rolling window features
window_features = RollingFeatures(
    stats=['mean', 'std'],
    window_sizes=7
)

# Create a recursive forecaster
forecaster = ForecasterRecursive(
    estimator=xgboost_model,
    window_features=window_features
)

# Create the "cv" with a single fold
cv = TimeSeriesFold(
    initial_train_size = len(time_series_train),
    steps = len(time_series_test)
)

# Dictionary of parameters to test
param_distributions = { 
    "max_depth": randint(1, 2),
    "n_estimators": randint(50, 300),
    "learning_rate": uniform(0.01, 0.2),
    "subsample": uniform(0.6, 0.4)
}

lags_grid = [
    [1, 7, 30]#,
    #[1, 7, 30, 90],
    #[1, 7, 30, 90, 120],
    #[90, 120],
    #[30, 90, 120],
]

# Random Search
results = random_search_forecaster(
    forecaster = forecaster,
    y = time_series,
    cv = cv,
    param_distributions = param_distributions,
    metric = 'mean_absolute_error',
    n_iter = 2,                         #the best parameters were found using 1000 iterations
    random_state = 30,
    return_best = True,
    verbose = False,
    exog = exog_feat_df,
    lags_grid=lags_grid,
    suppress_warnings=True
)

In [0]:
#overview on results dataframe
results[['mean_absolute_error','lags', 'subsample', 'max_depth', 'learning_rate', 'n_estimators']]

In [0]:
#BEST PARAMETERS FROM TUNING, WITH TRANSACTIONS
lags_wt = [1, 7, 30, 90, 120]
n_estim_wt = 67
max_dep_wt = 1
learn_rate_wt = 0.0748
subsamp_wt = 0.61


#BEST PARAMETERS FROM TUNING, WITHOUT TRANSACTIONS
lags_wot = [1, 7, 30, 90, 120]
n_estim_wot = 54
max_dep_wot = 1
learn_rate_wot = 0.153
subsamp_wot = 0.9

##Model / with Transactions

In [0]:
# Define the XGBoost regressor model 
xgboost_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators = n_estim_wt,             # here it uses the best value from the tuning, i.e. 54
    max_depth = max_dep_wt,                # here it uses the best value from the tuning, i.e. 1
    learning_rate = learn_rate_wt,         # here it uses the best value from the tuning, i.e. 0.153
    subsample = subsamp_wt,                # here it uses the best value from the tuning, i.e. 0.9
    random_state = 30
)

# Set up rolling window features
window_features = RollingFeatures(
    stats=['mean', 'std'],
    window_sizes=7
)

# Create a recursive forecaster
forecaster = ForecasterRecursive(
    estimator=xgboost_model,
    lags=lags_wt,                          # here it's using the value from the tuning, i.e. [1, 7, 30, 90, 120]
    window_features=window_features
)

# Fit the forecaster on the training time series and exogenous features
forecaster.fit(
    y=time_series_train,
    exog=exog_feat_df_train
)

# Prediction for the whole time series using the known data for lags and rolling statistics
# This is useful to evaluate the model and check for overfitting
time_series_pred = backtesting_forecaster(
    forecaster=forecaster,
    y=time_series,
    cv=TimeSeriesFold(steps=1),
    metric=[],       
    exog=exog_feat_df,
    verbose=False,
    show_progress=False
)
time_series_pred = time_series_pred[1].drop(columns='fold')

# Print metrics for train and test set
print('model / with transactions')
model_evaluate(time_series, time_series_pred)

In [0]:
#   	  train			        test
#r2	      0.8020322572677951	0.4206263402550948
#rmse	  4777.744938742869	    9187.369990158511
#mae	  3437.858614561988	    6185.573756250001

In [0]:
# Determine the number of steps to forecast (length of the test set)
test_size = len(time_series_test)

# Generate predictions for the test period with recursive model which uses predicted data for lags and rolling statisticss 
time_series_test_pred = forecaster.predict(
    steps=test_size,
    exog=exog_feat_df_test
)

# printing metrics for the forecasted period using recursive model
forecast_evaluate(time_series_test, time_series_test_pred)

# showing results on the plot
plt.figure(figsize=(20, 5))
plt.plot(time_series, label="real")
plt.plot(time_series_test_pred, label="pred")
plt.title("time series forecast WITH transactions")
plt.legend()
plt.show()

In [0]:
#results without transactions
#r2:    0.21693280683578042
#rmse:  10680.986047920094
#mae:   7813.880541319445

**Featured Used**
* day of month
* cyclical encoded weekday, month, doy
* time index
* is_weekend
* lags = [1, 7, 30, 90, 120]
* rolling_mean_7, rolling_std_7


##Model / without Transactions

In [0]:
#time series train/test splitting
time_series = time_series.drop(columns=['transactions'])
time_series_train =time_series_train.drop(columns=['transactions'])
time_series_test =time_series_test.drop(columns=['transactions'])

exog_feat_df =exog_feat_df.drop(columns=['transactions'])
exog_feat_df_train = exog_feat_df_train.drop(columns=['transactions'])
exog_feat_df_test = exog_feat_df_test.drop(columns=['transactions'])

In [0]:
# Define the XGBoost regressor model 
xgboost_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators = n_estim_wot,             # here it uses the best value from the tuning, i.e. 54
    max_depth = max_dep_wot,                # here it uses the best value from the tuning, i.e. 1
    learning_rate = learn_rate_wot,         # here it uses the best value from the tuning, i.e. 0.153
    subsample = subsamp_wot,                # here it uses the best value from the tuning, i.e. 0.9
    random_state = 30
)

# Set up rolling window features
window_features = RollingFeatures(
    stats=['mean', 'std'],
    window_sizes=7
)

# Create a recursive forecaster
forecaster = ForecasterRecursive(
    estimator=xgboost_model,
    lags=lags_wot,                          # here it's using the value from the tuning, i.e. [1, 7, 30, 90, 120]
    window_features=window_features
)

# Fit the forecaster on the training time series and exogenous features
forecaster.fit(
    y=time_series_train,
    exog=exog_feat_df_train
)

# Prediction for the whole time series using the known data for lags and rolling statistics
# This is useful to evaluate the model and check for overfitting
time_series_pred = backtesting_forecaster(
    forecaster=forecaster,
    y=time_series,
    cv=TimeSeriesFold(steps=1),
    metric=[],       
    exog=exog_feat_df,
    verbose=False,
    show_progress=False
)
time_series_pred = time_series_pred[1].drop(columns='fold')

# Print metrics for train and test set
print('model / without transactions')
model_evaluate(time_series, time_series_pred)

In [0]:
#   model / with transactions
#	    train			        test
#r2	    0.8220489253329964	    0.31690986447828495
#rmse	4529.768976493147	    9975.87135283112
#mae	3265.9525966956967	    8105.769630555555

In [0]:
# Determine the number of steps to forecast (length of the test set)
test_size = len(time_series_test)

# Generate predictions for the test period with recursive model which uses predicted data for lags and rolling statisticss 
time_series_test_pred = forecaster.predict(
    steps=test_size,
    exog=exog_feat_df_test
)

# printing metrics for the forecasted period using recursive model
forecast_evaluate(time_series_test, time_series_test_pred)

# showing results on the plot
plt.figure(figsize=(20, 5))
plt.plot(time_series, label="real")
plt.plot(time_series_test_pred, label="pred")
plt.title("time series forecast WITHOUT transactions")
plt.legend()
plt.show()

In [0]:
#results with transaction
#r2:  0.2257888644131798
#rmse:  10620.416281012645
#mae:  8808.538337152777

**Featured Used**
* day of month
* cyclical encoded weekday, month, doy
* time index
* is_weekend
* lags = [1, 7, 30, 90, 120]
* rolling_mean_7, rolling_std_7
